In [1]:
# import the functions needed for validate and comparsion

import numpy as np
import cupy as cp
import pandas as pd


#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skmultiflow.data import FileStream
from sklearn.neural_network import MLPClassifier


# using plotly for plots
#import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
from fires_prints import FIRES
from stability import stability_factor

In [3]:
# Human Activity Recognition
# labels changed from [1,...,6] to [0,...,5]
# rows shuffled
# split into train set with 7352 instances and test set with 2948
stream = FileStream('datasets/Multiclass/har_train.csv', target_idx = 561)
dataset_name = "har"
n_selected_ftr = 100
n_window = 10
batch_size = 50
weights = None

In [4]:
stream.restart()
predictor = MLPClassifier()
x,y = stream.next_sample(batch_size=batch_size)
predictor.partial_fit(x,y, stream.target_values)

MLPClassifier()

In [5]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=stream.target_values,
                    mu_init=0,
                    sigma_init=1,
                    model='softmax',
                    class_probabilities=weights,
                    lr_mu=0.2,
                    lr_sigma=0.2,
                    number_monte_carlo_samples=100)               

In [6]:
fires_cuda_accuracy = []

fires_cuda_selected_ftrs = []
fires_cuda_stability = []

In [8]:



#for i in range(1):
# Load a new sample
x, y = stream.next_sample(batch_size=batch_size)
# Select features
ftr_weights = fires_model.weigh_features(x,y)
ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]
if np.isnan(ftr_weights).any():
    print("NAN occured")


# Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
x_reduced = np.zeros(x.shape)
x_reduced[:, ftr_selection] = x[:, ftr_selection]

# stability test
ftr_array = np.zeros(stream.n_features)
ftr_array[ftr_selection] = 1
fires_cuda_selected_ftrs.append(ftr_array)

if len(fires_cuda_selected_ftrs) >= 10:
    stability = stability_factor(fires_cuda_selected_ftrs[-10:])
    fires_cuda_stability.append(stability)


# Test
y_pred = predictor.predict(x_reduced)
    
fires_cuda_accuracy.append(accuracy_score(y, y_pred))


# Train
predictor.partial_fit(x_reduced, y)





one call
50
For class 0 there are 7 obs
r shape(7, 100, 561, 6)
4.99406718420424
-5.148857807929787
theta:
4.99464886922113
-5.146656279449741
theta times x
(7, 100, 6)
52.637543080593105
-48.675118611049314
-6.77259777908707
mean:
(7, 100)
18.144282156674958
-17.217085223567775
median:
(7, 100)
19.900315934872637
-18.452921721448426
a shape (7, 100)
6.77259777908707
-52.637543080593105
eta:
(7, 100, 6)
False
0.0
-95.58881487186002
exp eta:
1.0
3.064115766270318e-42
eta_sum
(7, 100)
2.546123026708787
1.0
softmax_all
(7, 100, 6)
1.0
3.0641157662703176e-42
False
(7, 100)
marginal
(7,)
0.23033944789180325
0.11189535486728801
False
softmax_derivative
(7, 100, 561, 6)
0.24864123768479954
-0.24905290534495508
False
0.005339267259457211
1.0016874888845615
For class 1 there are 4 obs
r shape(4, 100, 561, 6)
5.250338279504917
-4.545697888686622
theta:
5.249874314909863
-4.5452318498091415
theta times x
(4, 100, 6)
50.790665919955245
-51.8103122416785
-2.9393159398170834
mean:
(4, 100)
18.486980

MLPClassifier()

In [ ]:
ftr_weights


In [ ]:
np.exp(-750)

In [ ]:
fires_moving_average = pd.Series(fires_cuda_accuracy).rolling(window=n_window).mean().iloc[n_window-1:].value

In [11]:
test = ("iaeiae {} iaeiae "
        "iaeiae {} iaeia "
        "iaeiae").format(2,3)

In [12]:
test


'iaeiae 2 iaeiae iaeiae 3 iaeia iaeiae'

In [13]:
msg = ("The marginal likelihood of class {} "
                                   "reached 0. This will cause the feature "
                                   "weights to become NaN. Try to normalize "
                                   "your data or reduce learning rate."
                                   ).format("iaeiae")

In [14]:
msg

'The marginal likelihood of class iaeiae reached 0. This will cause the feature weights to become NaN. Try to normalize your data or reduce learning rate.'

In [17]:
0 in np.array((np.exp(-500),0.3, 1))

False